In [6]:
from app.main.models import EducationStaff
import requests
from config import ApeksAPI
from app.main.func import db_filter_req, db_request
from datetime import date
from calendar import monthrange

class LoadReport(EducationStaff):
    def __init__(self, year, month, department_id):
        super().__init__(year, month, department_id)
        self.lessons = self.get_lessons()

    def get_lessons(self):
        """List of lessons for selected month"""
        first_day = 1
        last_day = monthrange(self.year, self.month)[1]

        params = {
            "token": ApeksAPI.TOKEN,
            "table": 'schedule_day_schedule_lessons',
            "filter": "date between '" + date(self.year, self.month, first_day).isoformat() + "' and '" + date(self.year, self.month, last_day).isoformat() + "'"
        }
        resp = requests.get(
            ApeksAPI.URL + "/api/call/system-database/get", params=params)
        return resp.json()['data']



In [15]:
load = LoadReport(2021, 12, 12)

In [19]:
staff = load.staff_list()

In [18]:
lessons = load.get_lessons()

In [20]:
for lesson in lessons:
    lesson_staff = db_filter_req('schedule_day_schedule_lessons_staff', 'lesson_id', lesson.get('id'))
    for i in lesson_staff:
        if i['staff_id'] in staff:
            lesson['staff_id'] = i['staff_id']
        else:
            lessons.remove(lesson)

ValueError: list.remove(x): x not in list